In [1]:
from matplotlib import pyplot as plt
import numpy as np
import importlib
from simulator.ideal import SimulatorIdeal
from simulator.lennard import SimulatorLennard
from simulator.magnetic import SimulatorMagnetic

from multiprocessing import Pool, cpu_count

import logging.config
import logging
import sys
logging.config.fileConfig("/home/amatevosyan/telegramLogConfig")
logger = logging.getLogger("telegram")

In [7]:
pool = Pool(cpu_count()); pool

<multiprocessing.pool.Pool state=RUN pool_size=12>

In [10]:
params_model = {
    "group_name" : "Ensemble 3",
    "R" : 1.0,
    "Rz" : 0.1,
    "eccentricity": 0.3,
    "sigma":0.5,
    "epsilon":1.0,
    # "logger" : logger
}
params_init = {
    "energy": 1.0,
    "sigma_grid":0.55,
    "position_random_shift_percentage": 1.0/100,
    "planar": False,
    "zero_momentum": False,
}
params_simulation = {
    "iteration_time" : 500,
    "dt" : 1e-4,
    "record_interval" : 1e-2,
    "algorithm" : "VERLET"
}


In [11]:
sim1 = SimulatorMagnetic(Bz=1, **params_model)
sim1.init_positions_velocities(**params_init)
sim1.rotational_push(0.5)
sim1.angular_momentum(sim1.r_init, sim1.v_init)[2].sum()

12.20282130695872

In [5]:
for i in range(500):
    name = str(np.random.randint(1e10))
    
    sim1 = SimulatorMagnetic(Bz=0, name=name, **params_model)
    sim1.init_positions_velocities(**params_init)
    sim1.rotational_push(0.4)
    r_init, v_init = sim1.r_init, sim1.v_init
#     print("N =", sim1.particle_number())
    res1 = pool.apply_async(sim1.simulate_async, kwds=params_simulation)

    sim2 = SimulatorMagnetic(Bz=1, name=name, **params_model)
    sim2.r_init, sim2.v_init = r_init, v_init
    res2 = pool.apply_async(sim2.simulate_async,kwds=params_simulation)
    
    sim3 = SimulatorMagnetic(Bz=100, name=name, **params_model)
    sim3.r_init, sim3.v_init = r_init, v_init
    res3 = pool.apply_async(sim3.simulate_async,kwds=params_simulation)

10.799878354968241

In [13]:
sim1.norm(sim1.r_init[:2, 0])

1.2298373876248845